Comparativa entre simulación (sin defectos) y pieza real (puede tener deefctos)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [10]:
!python -m pip install torchvision
!python -m pip install opencv-python


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/39.5 MB ? eta -:--:--
     -                                        1.5/39.5 MB 31.8 MB/s eta 0:00:02
     ---                                      3.7/39.5 MB 46.9 MB/s eta 0:00:01
     -----                                    5.1/39.5 MB 36.0 MB/s eta 0:00:01
     ------                                   6.2/39.5 MB 33.0 MB/s eta 0:00:02
     -------                                  7.1/39.5 MB 32.5 MB/s eta 0:00:01
     -------                                  7.7/39.5 MB 27.3 MB/s eta 0:00:02
     --------                                 8.3/39.5 MB 26.5 MB/s eta 0:00:02
     ---------                                8.9/39.5 MB 23.7 MB/s eta 0:00:02
     ---------                                9.5/39.5 MB 22.6 MB/s eta 0:00:02
     ---------                               10.1/39.5 MB 22.2 MB/s eta 0:00:02
     ----------                              10.6/39.5 MB 21.9 MB/s eta 0:00:02
     -----------                             11.


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Definimos una función para cargar las imágenes 

def cargar_imagenes(ruta):
  contenedor=[]
  for archivo in os.listdir(ruta):
    ruta_completa=os.path.join(ruta, archivo)
    if os.path.isfile(ruta_completa):
      imagen=Image.open(ruta_completa)
      contenedor.append(imagen)
    return contenedor

In [ ]:
IMG_size=256
BATCH_SIZE = 32
EPOCHS = 30
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Definimos transformaciones sobre imágenes (rezise y normalización)

transform=transforms.Compose([transforms.Resize((IMG_size, IMG_size)),
                              transforms.ToTensor()
])

# Vamos a definir un autoencoder para hacer la comparativa entre imágenes

class Autoencoder3D(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 8, kernel_size=3, stride=2, padding=1), nn.ReLU(),
            nn.Conv3d(8, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
            nn.Conv3d(16, 32, kernel_size=3, stride=2, padding=1), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1), nn.ReLU(),
            nn.ConvTranspose3d(16, 8, kernel_size=3, stride=2, padding=1, output_padding=1), nn.ReLU(),
            nn.ConvTranspose3d(8, 1, kernel_size=3, stride=2, padding=1, output_padding=1), nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))


In [ ]:
from PIL import Image
import cv2
import os

def load_stack(folder, img_size=(128, 128)):
    files = sorted(os.listdir(folder))[:500]  # asegúrate de ordenarlas bien
    stack = []
    for file in files:
        img = cv2.imread(os.path.join(folder, file), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, img_size)
        stack.append(img.astype("float32") / 255.0)
    volume = np.stack(stack)  # shape: (depth, height, width)
    return volume

volume = load_stack("mi_carpeta")  # shape: (depth, h, w)
input_tensor = torch.tensor(volume).unsqueeze(0).unsqueeze(0)

# --- Instancia el modelo, loss y optimizador
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Autoencoder3D().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# --- Ejemplo de función de entrenamiento para 1 epoch
def train_one_epoch(dataloader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0
    for batch in dataloader:
        inputs = batch.to(device)  # batch shape: (batch_size, 3, depth, height, width)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)  # reconstrucción vs original
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(dataloader.dataset)

# --- Ejemplo uso con DataLoader (tú debes crear el dataset y loader)
# from torch.utils.data import DataLoader
# train_loader = DataLoader(mi_dataset, batch_size=1, shuffle=True)

# for epoch in range(30):
#     loss = train_one_epoch(train_loader, model, criterion, optimizer, device)
#     print(f"Epoch {epoch+1}, Loss: {loss:.4f}")